In [1]:
# 一通り動かしてみる

#ライブラリの自動インポート
%reload_ext autoreload
%autoreload 2

In [33]:
import pandas as pd
import openai
import json
from src.utils.prompt import prompt_ask_for_reason
from src.utils.GPT import json_generate,mask_target_value

In [13]:
#api_key
from api_key import api_key
openai.api_key = api_key

In [4]:
#laod dataset as dict
csv_path="dataset/BradleyMeltingPointDataset_practice.csv"
df=pd.read_csv(csv_path)
chemical_records=df.to_dict(orient='records')
chemical_records[:3]

[{'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar'},
 {'mpC': 33.5,
  'name': '1-naphthalenecarboxaldehyde',
  'smiles': 'O=Cc2cccc1ccccc12',
  'csid': 5960,
  'link': 'http://www.srcinc.com/what-we-do/product.aspx?id=133',
  'source': 'PHYSPROP'},
 {'mpC': 214.0,
  'name': 'tabernanthine',
  'smiles': 'COc4cc5nc1c(CCN3CC2CC1C3C(C2)CC)c5cc4',
  'csid': 4265369,
  'link': 'http://www.srcinc.com/what-we-do/product.aspx?id=133',
  'source': 'PHYSPROP'}]

In [29]:
record=chemical_records[0]
qa_gen_command="""
Provide the quantitative reasons within 300 words so that a scientist, who does not know the melting point, can predict the value.
We must quantitatively consider how the melting point shifts.
I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler aromatic compounds, contributing to a higher melting point. (qualitative discussion is practically meaningless!!!)
## Therefore, the compound has a melting point of 110°C (Never include the answer in the reason!!).

#Good example reasons
## Benzene has a boiling point of 80 degrees, but toluene with a methyl group improves the boiling point by about +30 degrees due to its larger molecular weight and other reasons (+30 contribution).
## Butane has a boiling point of -1°C, but butanol with a hydroxy group has a boiling point increase of about 115°C due to the strong influence of hydrogen bonding (+115 contribution).

#Output: Reason key
"""
prompt=prompt_ask_for_reason(name=record['name'],value=record["mpC"],
                      smiles=record['smiles'],command=qa_gen_command)
print(prompt)


Provide the quantitative reasons within 300 words so that a scientist, who does not know the melting point, can predict the value.
We must quantitatively consider how the melting point shifts.
I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler aromatic compounds, contributing to a higher melting point. (qualitative discussion is practically meaningless!!!)
## Therefore, the compound has a melting point of 110°C (Never include the answer in the reason!!).

#Good example reasons
## Benzene has a boiling point of 80 degrees, but toluene with a methyl group improves the boiling point by about +30 degrees due to its larger molecular weight and other reasons (+30 contribution).
## Butane has a boiling point of -1°C, but butanol with a hydroxy group has a boiling point increase of about 115°C due to the strong influence of hydrogen bonding (+115 contribution).

#Output: Reason key

Name: 3-phthalimidopropionic acid
S

In [30]:
res=json_generate(prompt=prompt)

In [35]:
mask_target_value(res["Reason"],record["mpC"])

'3-phthalimidopropionic acid has a predicted melting point of approximately [MASK]°C due to the presence of the phthalimide group, which is known to increase the melting point of compounds by approximately +40°C based on similar structures. The propionic acid moiety also contributes to the higher melting point through intermolecular interactions, resulting in an additional estimated increase of +20°C. Therefore, the combined structural features of the compound suggest a significant elevation in melting point, supporting the prediction of a value around [MASK]°C.'